In [3]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("American-Airlines") \
    .getOrCreate()

# Path to your CSV file
csv_file_path = "/Users/mika/ML-Big-Data/Project/Dataset/2008.csv"

# Read the CSV file into a DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df.show(5)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2008|    1|         3|        4|   2003|      1955|   2211|      2225|           WN|      335

In [5]:
df.count()

7009728

In [13]:
df = df.drop("Year")

In [14]:
df.show(5)

+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|    1|         3|        4|   2003|      1955|   2211|      2225|           WN|      335| N712SW|           

In [28]:
df.groupBy("DayofMonth").count().show()
df.groupBy("Day").count().show()

+----------+------+
|DayofMonth| count|
+----------+------+
|        31|127968|
|        28|229613|
|        26|226590|
|        27|222876|
|        12|227247|
|        22|230557|
|         1|229292|
|        13|227233|
|         6|228117|
|        16|229085|
|         3|233259|
|        20|229165|
|         5|228156|
|        19|227487|
|        15|230379|
|         9|227064|
|        17|233202|
|         4|228649|
|         8|229578|
|        23|229039|
+----------+------+


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Day` cannot be resolved. Did you mean one of the following? [`Dest`, `ArrDelay`, `DepDelay`, `Month`, `NASDelay`].;
'Aggregate ['Day], ['Day, count(1) AS count#6550L]
+- Project [Month#2359, DayofMonth#2360, DepTime#2362, CRSDepTime#2363, ArrTime#2364, CRSArrTime#2365, UniqueCarrier#2366, FlightNum#2367, TailNum#2368, ActualElapsedTime#2369, CRSElapsedTime#2370, AirTime#2371, ArrDelay#2372, DepDelay#2373, Origin#2374, Dest#2375, Distance#2376, TaxiIn#2377, TaxiOut#2378, Cancelled#2379, CancellationCode#2380, Diverted#2381, CarrierDelay#2382, WeatherDelay#2383, ... 3 more fields]
   +- Project [Month#2359, DayofMonth#2360, DayOfWeek#2361, DepTime#2362, CRSDepTime#2363, ArrTime#2364, CRSArrTime#2365, UniqueCarrier#2366, FlightNum#2367, TailNum#2368, ActualElapsedTime#2369, CRSElapsedTime#2370, AirTime#2371, ArrDelay#2372, DepDelay#2373, Origin#2374, Dest#2375, Distance#2376, TaxiIn#2377, Taxi

In [26]:
df = df.drop("DayOfWeek")

In [27]:
df.show(5)

+-----+----------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Month|DayofMonth|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+-----+----------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|    1|         3|   2003|      1955|   2211|      2225|           WN|      335| N712SW|              128|           150|    116|     -14| 